In [45]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

import mlflow
mlflow.set_experiment("Mlflow test 1")

<Experiment: artifact_location='file:///c:/Users/petit/Openclassrooms/P7/mlruns/498723390706114045', creation_time=1740494719252, experiment_id='498723390706114045', last_update_time=1740494719252, lifecycle_stage='active', name='Mlflow test 1', tags={}>

In [46]:
model_name = ""

In [47]:
df = pd.read_csv('preprocessed_tweet.csv')
df.head()

,Target,Tweet_preprocessed,Tweet_preprocessed_dl
0,0,awww thats a bummer you shoulda got david car...,"awww, that's a bummer. you shoulda got david..."
1,0,is upset that he cant update his facebook by t...,is upset that he can't update his facebook by ...
2,0,i dived many times for the ball managed to sav...,i dived many times for the ball. managed to sa...
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,no its not behaving at all im mad why am i her...,"no, it's not behaving at all. i'm mad. why am ..."


In [48]:
y = df['Target'].copy()
X = df['Tweet_preprocessed'].copy()

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [50]:
X_train.head()

612396     hey splogin connect speed should be ok ram may...
1445587    again wit the socks 2day huh hehe i like them ...
1310255               hey chick how you feeling this morning
351809       i feel like i never chill with you guys anymore
204527                               yep thats what happened
Name: Tweet_preprocessed, dtype: object

In [51]:
X_train[X_train.isna()]

Series([], Name: Tweet_preprocessed, dtype: object)

In [52]:
vectorizer_name = "tfidf"

vectorizer_params = {"stop_words": "english", "max_df": 0.95, "min_df": 1}

vectorizers = {
    'count-vectorizer': CountVectorizer(**vectorizer_params),
    'tfidf': TfidfVectorizer(**vectorizer_params)
}

vectorizer = vectorizers[vectorizer_name]

vectorizer.fit(X_train)

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [53]:
#count_vectorizer = CountVectorizer(stop_words='english', max_df=0.95, min_df=1)
#tfidf = TfidfVectorizer(stop_words='english', max_df=0.95, min_df=1)

#count_vectorizer.fit(X_train)

#X_train = count_vectorizer.transform(X_train)
#X_test = count_vectorizer.transform(X_test)

In [54]:
model = LogisticRegression(max_iter=1000)

#model.fit(X_train, y_train)

#y_pred = model.predict(X_test)

accuracy_score(y_test, y_pred)

0.7749247174868553

In [55]:
from time import perf_counter

In [56]:
with mlflow.start_run():
    mlflow.log_params({"vectorizer": vectorizer_name})
    
    start = perf_counter()

    model.fit(X_train, y_train)
    
    
    y_pred = model.predict(X_test)
    
    end = perf_counter()
    
    duration = round(end - start, 2)
    
    score = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", score)
    mlflow.log_metric("duration", duration)